In [4]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

import json

import urllib
from sqlalchemy import create_engine

import pandas as pd
import numpy as np

import datetime

In [5]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

In [6]:
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"

# desktop client
# client_secrets_file = "<client_secret_file.json>"

#api key
api_key = "<API Key>"

# # Get credentials and create an API client
# flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
#     client_secrets_file, scopes)

# # credentials = flow.run_console()
# youtube = googleapiclient.discovery.build(
#     api_service_name, api_version, credentials=credentials)

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=api_key)

In [7]:
countries = ['CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU', 'US']

In [8]:
def get_video_categories(country_code):
    request = youtube.videoCategories().list(
        part="snippet",
        regionCode=country_code
    )
    return request.execute()

response = get_video_categories('US')
print(json.dumps(response, indent=4))

{
    "kind": "youtube#videoCategoryListResponse",
    "etag": "QteLrrS_X7rM7rlcU_e7qa0embQ",
    "items": [
        {
            "kind": "youtube#videoCategory",
            "etag": "grPOPYEUUZN3ltuDUGEWlrTR90U",
            "id": "1",
            "snippet": {
                "title": "Film & Animation",
                "assignable": true,
                "channelId": "UCBR8-60-B28hp2BmDPdntcQ"
            }
        },
        {
            "kind": "youtube#videoCategory",
            "etag": "Q0xgUf8BFM8rW3W0R9wNq809xyA",
            "id": "2",
            "snippet": {
                "title": "Autos & Vehicles",
                "assignable": true,
                "channelId": "UCBR8-60-B28hp2BmDPdntcQ"
            }
        },
        {
            "kind": "youtube#videoCategory",
            "etag": "qnpwjh5QlWM5hrnZCvHisquztC4",
            "id": "10",
            "snippet": {
                "title": "Music",
                "assignable": true,
                "channelId": "UCBR

In [6]:
def create_category_table(response):
    df = pd.json_normalize(response['items'])

    df_new = pd.DataFrame()
    df_new['id'] = df['id'].astype('int')
    df_new['title'] = df['snippet.title']

    df_new = df_new.set_index('id')
    return df_new

categories = create_category_table(response)
categories

,title
id,
1,Film & Animation
2,Autos & Vehicles
10,Music
15,Pets & Animals
17,Sports
18,Short Movies
19,Travel & Events
20,Gaming
21,Videoblogging


In [7]:
def get_video_list(country_code):

    request = youtube.videos().list(
        part="snippet,statistics",
        chart="mostPopular",
        regionCode=country_code,
        maxResults=50,
    )
    response = request.execute()
    final_response = {'items': response['items']}
#     print(reponse)

    while "nextPageToken" in response.keys():
        request = youtube.videos().list(
            part="snippet,statistics",
            chart="mostPopular",
            regionCode=country_code,
            maxResults=50,
            pageToken=response['nextPageToken']
        )
        response = request.execute()
        final_response['items'] += response['items']

    return final_response

response = get_video_list('US')
response

{'items': [{'kind': 'youtube#video',
   'etag': 'Q9mgw3KOdrqcJLjKdgPeKD3n-PU',
   'id': 'nOI67IDlNMQ',
   'snippet': {'publishedAt': '2023-03-24T03:59:08Z',
    'channelId': 'UC3IZKseVpdzPSBaWxBxundA',
    'title': "지민 (Jimin) 'Like Crazy' Official MV",
    'description': "지민 (Jimin) 'Like Crazy' Official MV\n\n'FACE' Release\n📆 3/24 1PM KST | 0AM ET\n💽 Listen to 'FACE': https://ingrv.es/face\n\nCredits:  \nProduction: OUI PRODUCTION\nDirector: Oui Kim\nProducer: Yeonjin Kim, Hyunsik Bang\n1st AD: Huiwon Song\n2nd AD: Sunyoung Kim, Hyeri Shin\n\nDirector of Photography: EumKo\nB Cam Operator: Hyungyun Kim\nFocus Puller: Youngwoo Lee, Heejung Kim\n2nd AC: Eunil Lee, Hyungjun Ahn\nData Manager: Dongwon Choi\n3rd AC: Donghyun Lee\n\nJimmy Jib Operator: Youngjung Kim\nJimmy Jib Assistant: Hyunin Kim, Hojun Choi, Minsu Kim\n\nGrip: Jaekeun Han, Jumyung Lee, Hyunseok Jo\n\nGaffer: Joonghyuk Jung\nCrew: Hyunsuk Kim, Minhyeok Back, Hwayong Chung, Dongyun Sin, Yusung Han, Youngmin Choi\n\nProdu

In [8]:
def create_video_list_table(response, country_code):
    df = pd.json_normalize(response['items'])
    df_new = pd.DataFrame()
    
    df_new['id'] = df['id']
    df_new['title'] = df['snippet.title']
    df_new['channel'] = df['snippet.channelTitle']
    df_new['channel_id'] = df['snippet.channelId']
    df_new['description'] = df['snippet.description']
    df_new['thumbnail'] = df['snippet.thumbnails.default.url']
    df_new['category'] = df['snippet.categoryId'].astype('int')
    df_new['view_count'] = df['statistics.viewCount'].astype('int')
    df_new['like_count'] = df['statistics.likeCount'].astype('Int64')
    df_new['comment_count'] = df['statistics.commentCount'].astype('Int64')
    df_new['publish_date'] = pd.to_datetime(df['snippet.publishedAt'])
    
    df_new['trending_date'] = pd.to_datetime('today').normalize()
    
    df_new['country'] = country_code
    
    df_new = df_new.reset_index()
    df_new = df_new.rename(columns={"index":"rank"})
    df_new['rank'] = df_new.index + 1
    
    df_new = df_new.set_index('id')
    
    return df_new

video_list = create_video_list_table(response, 'US')
video_list

,rank,title,channel,channel_id,description,thumbnail,category,view_count,like_count,comment_count,publish_date,trending_date,country
id,,,,,,,,,,,,,
nOI67IDlNMQ,1,지민 (Jimin) 'Like Crazy' Official MV,HYBE LABELS,UC3IZKseVpdzPSBaWxBxundA,지민 (Jimin) 'Like Crazy' Official MV\n\n'FACE' ...,https://i.ytimg.com/vi/nOI67IDlNMQ/default.jpg,10,8000626,1702506,153457,2023-03-24 03:59:08+00:00,2023-03-24,US
QXQQAsIhHMw,2,"ROSALÍA, Rauw Alejandro - BESO (Official Video)",RosaliaVEVO,UCy2bCk5KnIfjmWYFHMZcd5w,Official video for “BESO” by ROSALÍA & Rauw Al...,https://i.ytimg.com/vi/QXQQAsIhHMw/default.jpg,10,2476899,294622,7704,2023-03-24 04:00:06+00:00,2023-03-24,US
5vXiaaIxPHs,3,BTS's Jimin Talks About His Solo Album Face an...,The Tonight Show Starring Jimmy Fallon,UC8-Th83bH_thdKZDJCrn88g,"Jimin talks about BTS, meeting President Biden...",https://i.ytimg.com/vi/5vXiaaIxPHs/default.jpg,23,2575800,423436,16288,2023-03-24 04:19:06+00:00,2023-03-24,US
ntCZjb_AAWE,4,ARCANGEL || BZRP Music Sessions #54,Bizarrap,UCmS75G-98QihSusY7NfCZtw,ARCANGEL || BZRP Music Sessions #54\n\n\nLyric...,https://i.ytimg.com/vi/ntCZjb_AAWE/default.jpg,24,17674394,2007163,102383,2023-03-23 00:00:18+00:00,2023-03-24,US
MSSdmVe16mo,5,"Maluma & Anuel AA - Diablo, Qué Chimba (Offici...",Maluma,UClZuKq2m0Qu-HkopkSBLpEw,"Maluma & Anuel AA - Diablo, Qué Chimba (Offici...",https://i.ytimg.com/vi/MSSdmVe16mo/default.jpg,10,1422285,174841,8861,2023-03-23 22:00:09+00:00,2023-03-24,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...
oxo-loXdcH0,196,Bono and The Edge: Tiny Desk Concert,NPR Music,UC4eYXhJI4-7wSWc8UNRwD4A,"Robin Hilton | March 17, 2023\nIt's hard to ov...",https://i.ytimg.com/vi/oxo-loXdcH0/default.jpg,10,1246161,46924,3359,2023-03-17 16:00:10+00:00,2023-03-24,US
DMiV0FeIVUM,197,Puerto Rico vs Mexico Game Highlights | 2023 W...,MLB,UCoLrcjPV5PbUrUyXq5mjc_A,Team Puerto Rico takes on Team Mexico in the W...,https://i.ytimg.com/vi/DMiV0FeIVUM/default.jpg,17,2384194,34144,2636,2023-03-18 03:49:31+00:00,2023-03-24,US
sU6CXUBKurY,198,Russ - NASTY (Official Audio),Russ,UCiIETTq2It3OzW8_WYRPTyQ,"Russ - ""NASTY"" (Official Audio)\n\nhttps://rus...",https://i.ytimg.com/vi/sU6CXUBKurY/default.jpg,10,410764,21618,1215,2023-03-17 04:00:13+00:00,2023-03-24,US


In [9]:
username = 'root'
password = 'root'
host = '4.240.82.255'
db_name = 'youtube_db'

postgres_url = f'postgresql://{username}:{password}@{host}:5432/{db_name}'

engine = create_engine(postgres_url)

In [10]:
def to_db(categories, video_list, country_code):
    
    category_table_name = "videoCategories"
    table_name = "trendingVideos"
    
    
    try:
        categories.head(n=0).to_sql(name=category_table_name, con=engine, if_exists="replace")
    except ValueError:
        pass
    finally:
        categories.to_sql(name=category_table_name, con=engine, if_exists="append")
    
    
    try:
        video_list.head(n=0).to_sql(name=table_name, con=engine, if_exists="fail")
    except ValueError:
        pass
    finally:
        video_list.to_sql(name=table_name, con=engine, if_exists="append")

        
# to_db(categories, video_list, 'US')

In [11]:
for country in countries:
    response = get_video_categories(country)
    categories = create_category_table(response)
    
    response = get_video_list(country)
    video_list = create_video_list_table(response, country)
    
    to_db(categories, video_list, country)
    
    print(f"Uploaded for country {country}")
    

Uploaded for country CA
Uploaded for country DE
Uploaded for country FR
Uploaded for country GB
Uploaded for country IN
Uploaded for country JP
Uploaded for country KR
Uploaded for country MX
Uploaded for country RU
Uploaded for country US
